In [3]:
%load_ext autoreload
%autoreload 2

from util_0801 import *
from myalgorithm_0801_1 import algorithm

## 실험 내용
- 크루스칼 결과를 가중치마다 누적하되, 평균 비용이 높은 번들 n개만 해제하는 방식을 사용하면 어떨까?

In [4]:
problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_2.json'
# problem_file = '../alg_test_problems_20240429/TEST_K50_1.json'

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

for v in ALL_RIDERS:
    print(v)

print(K)

Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 20])
Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30])
Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 100])
100


### 초기 결과

In [12]:
## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## ------------------  크루스칼 함수   -----------------------------

def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            if avg_method == 'avg':
                dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
            elif avg_method == 'two_seq':
                dist1 = DIST[i][j]
                dist2 = DIST[i + K][j + K]
            elif avg_method == 'two':
                order_num1 = all_bundles[i].shop_seq[0]
                order_num2 = all_bundles[j].shop_seq[0]

                dist1 = DIST[order_num1][order_num2]
                dist2 = DIST[order_num1 + K][order_num2 + K]  
            else:
                assert False

            # weight1 = (dist1 + dist2) / 900

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(len(all_bundles)))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue

        new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    rider_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, rider_availables

## --------------- 초기 번들링 최적화 코드 --------------------------

to_checks = []

for weight1 in [0, 1, 2]:
    for weight2 in [-4, -3, -2, -1, 0]:
        avg_method = 'two'
        bundle_merging_function = try_merging_bundles_by_dist
        default_get_dist_function = get_dist_by_coords

        inf = float('inf')

        car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
        bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
        walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

        all_bundles = []
        for ord in ALL_ORDERS:
            new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
            car_rider.available_number -= 1
            all_bundles.append(new_bundle)

        # 2개 주문 묶음 생성
        all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

        # 4개 주문 묶음 생성
        all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

        to_check = [bundle for bundle in all_bundles if len(bundle.shop_seq) == 4]

        # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
        new_all_bundles = []
        for bundle in all_bundles:
            if len(bundle.shop_seq) >= 3:
                new_all_bundles.append(bundle)
            else:
                old_rider = bundle.rider
                old_rider.available_number += 1
                for order_num in bundle.shop_seq:
                    order = ALL_ORDERS[order_num]

                    new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                    car_rider.available_number -= 1
                    new_all_bundles.append(new_bundle)

        all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

        ## ------------------- 라이더 재배치 -------------------------------

        all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
        for rider_i in range(3):
            ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

        ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

        solution = [
                # rider type, shop_seq, dlv_seq
                [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
                for bundle in all_bundles
        ]

        with open(problem_file, 'r') as f:
            prob = json.load(f)

        K = prob['K']

        ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
        ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

        DIST = np.array(prob['DIST'])
        for r in ALL_RIDERS:
            r.T = np.round(DIST/r.speed + r.service_time).astype(int)

        checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

        to_checks.append((to_check, len(all_bundles), weight1, weight2, checked_solution['avg_cost']))

In [14]:
weights = []
for weight1 in [0, 1, 2]:
    for weight2 in [-4, -3, -2, -1, 0]:
        weights.append((weight1, weight2))

In [16]:
print(weights)

[(0, -4), (0, -3), (0, -2), (0, -1), (0, 0), (1, -4), (1, -3), (1, -2), (1, -1), (1, 0), (2, -4), (2, -3), (2, -2), (2, -1), (2, 0)]


### 크루스칼 코드 변형 결과 확인

In [41]:
## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

## ------------------  크루스칼 함수   -----------------------------

def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
    def find(v):
        while v != parent[v]:
            parent[v] = parent[parent[v]]
            v = parent[v]

        return v

    def union(a, b, new_bundle):
        if a > b:
            a, b = b, a

        parent[b] = a
        all_bundles[a] = new_bundle

    for i in range(len(all_bundles)):
        bundle = all_bundles[i]

        shop_seq = bundle.shop_seq

        xs_s_sum = 0
        ys_s_sum = 0

        xs_e_sum = 0
        ys_e_sum = 0

        readytimes_sum = 0
        deadlines_sum = 0

        shop_seq_len = len(shop_seq)

        for order_num in shop_seq:
            order = ALL_ORDERS[order_num]

            xs_s_sum += order.shop_lat
            ys_s_sum += order.shop_lon

            xs_e_sum += order.dlv_lat
            ys_e_sum += order.dlv_lon

            readytimes_sum += order.ready_time
            deadlines_sum += order.deadline

        xs_s_avg = xs_s_sum / shop_seq_len
        ys_s_avg = ys_s_sum / shop_seq_len

        xs_e_avg = xs_e_sum / shop_seq_len
        ys_e_avg = ys_e_sum / shop_seq_len

        readytimes_avg = readytimes_sum / shop_seq_len
        deadlines_avg = deadlines_sum / shop_seq_len

        avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

        bundle.avg_info = avg_info

    edges = []
    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            avg_info1 = all_bundles[i].avg_info
            avg_info2 = all_bundles[j].avg_info

            sx1, sy1, ex1, ey1, r1, d1 = avg_info1
            sx2, sy2, ex2, ey2, r2, d2 = avg_info2

            r_diff = abs(r1 - r2)
            d_diff = abs(d1 - d2)

            start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

            if avg_method == 'avg':
                dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
            elif avg_method == 'two_seq':
                dist1 = DIST[i][j]
                dist2 = DIST[i + K][j + K]
            elif avg_method == 'two':
                order_num1 = all_bundles[i].shop_seq[0]
                order_num2 = all_bundles[j].shop_seq[0]

                dist1 = DIST[order_num1][order_num2]
                dist2 = DIST[order_num1 + K][order_num2 + K]  
            else:
                assert False

            # weight1 = (dist1 + dist2) / 900

            diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2

            edges.append((i, j, diff_score))

    parent = list(range(len(all_bundles)))
    edges.sort(key=lambda x: x[2])

    for bundle_num1, bundle_num2, diff_score in edges:
        rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

        if rbn1 == rbn2:
            continue

        new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

        if new_bundle is not None:
            all_bundles[rbn1].rider.available_number += 1
            all_bundles[rbn2].rider.available_number += 1
            
            new_bundle.rider.available_number -= 1

            union(rbn1, rbn2, new_bundle)

    parent = [find(v) for v in parent]

    result_bundles = [all_bundles[v] for v in set(parent)]
    rider_availables = [rider.available_number for rider in ALL_RIDERS]

    return result_bundles, rider_availables

## --------------- 초기 번들링 최적화 코드 --------------------------

avg_method = 'two'
bundle_merging_function = try_merging_bundles_by_dist
default_get_dist_function = get_dist_by_coords

inf = float('inf')

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

for weight1, weight2 in [(0, -4), (0, -3), (0, -2), (0, -1), (0, 0), (1, -4), (1, -3), (1, -2), (1, -1), (1, 0), (2, -4), (2, -3), (2, -2), (2, -1), (2, 0)]:
    # 2개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

    # 4개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

    # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
    new_all_bundles = []
    for bundle in all_bundles:
        if len(bundle.shop_seq) >= 3:
            new_all_bundles.append(bundle)
        else:
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                new_all_bundles.append(new_bundle)

    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

    ## ------------------- 라이더 재배치 -------------------------------

    all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

    all_bundles.sort(key=lambda x: x.cost_per_ord)
    
    if not (weight1 == 2 and weight2 == 0):
        temp_bundles = []
        for _ in range(3):
            bundle = all_bundles.pop()
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                temp_bundles.append(new_bundle)

        all_bundles.extend(temp_bundles)


## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution)

{'total_cost': 369210.10000000003, 'avg_cost': 3692.1010000000006, 'num_drivers': 35, 'total_dist': 162954, 'feasible': True, 'infeasibility': None, 'bundles': [['BIKE', [10, 9, 15, 31], [15, 10, 9, 31]], ['BIKE', [78, 94, 50, 46], [46, 94, 50, 78]], ['BIKE', [33, 32, 18, 41], [18, 32, 33, 41]], ['BIKE', [42, 12, 64, 47], [12, 42, 47, 64]], ['BIKE', [11, 16, 30, 17], [17, 30, 16, 11]], ['CAR', [55, 68, 96, 93], [68, 93, 96, 55]], ['CAR', [54, 81, 58, 59], [54, 81, 58, 59]], ['CAR', [8, 1, 7, 14], [8, 14, 7, 1]], ['BIKE', [71, 90, 56, 77], [71, 90, 56, 77]], ['BIKE', [83, 84, 79, 44], [44, 83, 79, 84]], ['CAR', [65, 53, 36, 85], [36, 65, 53, 85]], ['CAR', [49, 63, 92, 74], [63, 49, 74, 92]], ['CAR', [35, 39, 24], [24, 35, 39]], ['BIKE', [34, 22, 73, 82], [22, 34, 73, 82]], ['BIKE', [48, 37, 45, 60], [37, 48, 45, 60]], ['BIKE', [40, 20, 29], [20, 29, 40]], ['CAR', [57, 88], [57, 88]], ['CAR', [86, 75], [86, 75]], ['CAR', [70, 43], [70, 43]], ['CAR', [19, 67], [19, 67]], ['CAR', [4, 52], 

In [23]:
a = all_bundles[0]

a.cost_per_ord

3605.3333333333335

In [25]:
all_bundles.sort(key=lambda x: x.cost_per_ord)

all_bundles

[Bundle(all_orders, BIKE, [10, 9, 15, 31], [15, 10, 9, 31], 80, True),
 Bundle(all_orders, BIKE, [78, 94, 50, 46], [46, 94, 50, 78], 93, True),
 Bundle(all_orders, BIKE, [33, 32, 18, 41], [18, 32, 33, 41], 83, True),
 Bundle(all_orders, BIKE, [42, 12, 64, 47], [12, 42, 47, 64], 99, True),
 Bundle(all_orders, BIKE, [11, 16, 30, 17], [17, 30, 16, 11], 88, True),
 Bundle(all_orders, CAR, [55, 68, 96, 93], [68, 93, 96, 55], 118, True),
 Bundle(all_orders, CAR, [54, 81, 58, 59], [54, 81, 58, 59], 175, True),
 Bundle(all_orders, CAR, [8, 1, 7, 14], [8, 14, 7, 1], 125, True),
 Bundle(all_orders, BIKE, [71, 90, 56, 77], [71, 90, 56, 77], 57, True),
 Bundle(all_orders, BIKE, [83, 84, 79, 44], [44, 83, 79, 84], 87, True),
 Bundle(all_orders, CAR, [87, 57, 88], [57, 88, 87], 110, True),
 Bundle(all_orders, CAR, [65, 53, 36, 85], [36, 65, 53, 85], 154, True),
 Bundle(all_orders, CAR, [4, 26, 52], [4, 26, 52], 73, True),
 Bundle(all_orders, CAR, [49, 63, 92, 74], [63, 49, 74, 92], 147, True),
 Bund